# Specifying a reaction network
$$
\def\n{\mathbf{n}}
\def\x{\mathbf{x}}
\def\N{\mathbb{\mathbb{N}}}
\def\X{\mathbb{X}}
\def\NX{\mathbb{\N_0^\X}}
\def\C{\mathcal{C}}
\def\Jc{\mathcal{J}_c}
\def\DM{\Delta M_{c,j}}
\newcommand\diff{\mathop{}\!\mathrm{d}}
\def\Xc{\mathbf{X}_c}
\def\Yc{\mathbf{Y}_c}
\newcommand{\muset}[1]{\dot{\{}#1\dot{\}}}
$$

In [50]:
# initialize sympy printing (for latex output)
from sympy import init_printing
init_printing()

# import functions and classes for compartment models
from compartments import *
from compartments_display import *

## Basics

The left-hand-side and right-hand-site of transitions are expressions involving zero or more `Compartments`.

To define a compartment $[\x]$, first create a content variable $\x \in \N_0^D$, then define a compartment $[\x]$ with content $\x$.

In [51]:
x = ContentVar('x')
Compartment(x)

Compartment(x)

Content variables are $D$-dimensional. In equations, `x[d]` can be used to denote the copy number of chemical specises `d`.

Next, we can build a stoichiometric equation using the `Transition` class, which takes two parameter: the reactants expression and the products expression. For empty reactants or products, use `EmptySet()`.
The following defines the "Exit" transition from the nested birth-death process example.

In [52]:
transition_E = Transition(Compartment(x), EmptySet(), name='E')
display(transition_E)

[x] ---> EmptySet())

`name="E"` is an optional parameter, defining the subscript used for the transition class. (It is only used for displaying equations, e.g., the $h_E$ over the arrow in the above example.)

To define a propensity for the transition class, we have to define the content-independent rate constant $k_c$, the reactant tuning function $g_c$, and the conditional probability $\pi_c$.

In [53]:
k_E = Constant('k_E')
g_E = 1
pi_E = pi_c_identity()

The type of $\pi_c$ is `Pi_c`, which is a class comprising
* an expression or symbol to use for displaying $\pi_c$ in compound expression
* a function `conditional_expectation` that takes an expression over reactants $\Xc$, and returns it's expectation over all product compartment variables.

There are generators for several predefined `Pi_c`. In the "Exit" transition example, we use `pi_c_identity()` which has symbol $1$ and returns the expression over reactants $\Xc$ as is. It is appropriate when $\Yc = \emptyset$ or, more precisely, if all content variables occurring in products already occur in reactants.

To specify a particular transition with its propensity, use the tuple `(transition_c, k_c, g_c, pi_c)`.

In [54]:
Exit = (transition_E, k_E, g_E, pi_E)

The function `display_propensity()` displays the assembled propensity function, where $w(\n;\Xc)$ is derived from the reactants in the stoichiometric equation.
The function `display_propensity_details()` displays the constituent expressions $k_c$, $g_c$, and $\pi_c$.

In [55]:
display_propensity(Exit)

In [56]:
display_propensity_details(Exit)

To specify a reaction network comprising multiple transitions, collect the in a list.
The function `display_transitions()` displays the equations and propensities of the network.

In [57]:
transitions = [Exit]
display_transitions(transitions)

[x] ---> EmptySet())

## Chemical Events

To specify chemical reactions, we need the `ContentChange()` class. It is constructed with $D$ (number of species) integer arguments, specifying the change (to a compartment variable) in each species.

For example, the "birth" transition from the nested birth-death process example can be defined as

In [58]:
x = ContentVar('x')
Transition(Compartment(x), Compartment(x + ContentChange(1)), name='b')

[x] ---> [(1,) + x])

Note, that `Compartment(...)` in the product encloses `x + ContentChange(1)`, so the `+` acts on the compartment content, rather than specifying a set of compartments.

Besides addition, also multiplication can be used inside compartment content equations, e.g.

In [59]:
Transition(Compartment(x), Compartment(2*x + ContentChange(1)))

[x] ---> [(1,) + 2*x])

## Compartment Events

To specify compartment events, simply construct (multi-)sets of compartments using `+`.

For example, a "Coagulation" transition can be specified as

In [60]:
x1 = ContentVar('x')
x2 = ContentVar('x\'')
Transition(Compartment(x1) + Compartment(x2), Compartment(x1 + x2), name='C')

[x] + [x'] ---> [x + x'])

In principle, also multiplication can be used, e.g., `2*Compartment(x1)` means "two compartments of content 'x1'".
But the moment derivation mechanics is not ready for that yet...

## Example: nested birth-death process

As a complete example, here is the definition of nested birth-death process.

In [61]:
D = 1 # number of species

y = ContentVar('y')
x = ContentVar('x')

# Intake
transition_I = Transition(EmptySet(), Compartment(y), name='I')
k_I = Constant('k_I')
g_I = 1
pi_I = pi_c_poisson(
    Symbol("\pi_I", positive=True),
    y[0],
    Symbol("\lambda", positive=True))

# Exit
transition_E = Transition(Compartment(x), EmptySet(), name='E')
k_E = Constant('k_E')
g_E = 1
pi_E = pi_c_identity()

# birth
transition_b = Transition(Compartment(x), Compartment(x + ContentChange(1)), name='b')
k_b = Constant('k_b')
g_b = 1
pi_b = pi_c_identity()

# death
transition_d = Transition(Compartment(x), Compartment(x + ContentChange(-1)), name='d')
k_d = Constant('k_d')
g_d = x[0] # TODO x should be enough here, in case D=1.
pi_d = pi_c_identity()

transitions = [
    (transition_I, k_I, g_I, pi_I),
    (transition_E, k_E, g_E, pi_E),
    (transition_b, k_b, g_b, pi_b),
    (transition_d, k_d, g_d, pi_d)
]

display_transitions(transitions)

EmptySet() ---> [y])

[x] ---> EmptySet())

[x] ---> [(1,) + x])

[x] ---> [(-1,) + x])